<a href="https://colab.research.google.com/github/AviralTripathim22ma012/Semantic-Segmentation/blob/main/U-net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import os
import cv2
from google.colab.patches import cv2_imshow
from torch.utils.data import Dataset,DataLoader
import torchvision.transforms as transforms

from torch.nn import functional as F

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install -q kaggle
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/Kaggle/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d balraj98/duts-saliency-detection-dataset

100% 391M/392M [00:12<00:00, 38.2MB/s]
100% 392M/392M [00:12<00:00, 32.2MB/s]


In [ ]:
!unzip /content/duts-saliency-detection-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: DUTS-TR/DUTS-TR-Mask/n03764736_2979.png  
  inflating: DUTS-TR/DUTS-TR-Mask/n03764736_2990.png  
  inflating: DUTS-TR/DUTS-TR-Mask/n03764736_3066.png  
  inflating: DUTS-TR/DUTS-TR-Mask/n03764736_3070.png  
  inflating: DUTS-TR/DUTS-TR-Mask/n03764736_3086.png  
  inflating: DUTS-TR/DUTS-TR-Mask/n03764736_3153.png  
  inflating: DUTS-TR/DUTS-TR-Mask/n03764736_3168.png  
  inflating: DUTS-TR/DUTS-TR-Mask/n03764736_3216.png  
  inflating: DUTS-TR/DUTS-TR-Mask/n03764736_3220.png  
  inflating: DUTS-TR/DUTS-TR-Mask/n03764736_3227.png  
  inflating: DUTS-TR/DUTS-TR-Mask/n03764736_3240.png  
  inflating: DUTS-TR/DUTS-TR-Mask/n03764736_3368.png  
  inflating: DUTS-TR/DUTS-TR-Mask/n03764736_3432.png  
  inflating: DUTS-TR/DUTS-TR-Mask/n03764736_3527.png  
  inflating: DUTS-TR/DUTS-TR-Mask/n03764736_3584.png  
  inflating: DUTS-TR/DUTS-TR-Mask/n03764736_3653.png  
  inflating: DUTS-TR/DUTS-TR-Mask/n03764736_3664.png  
  inflating: D

In [ ]:
data = pd.read_csv('/content/metadata.csv')
train_index = data.loc[data['split']=='train'].index
test_index = data.loc[data['split']=='test'].index

train_data = data.iloc[train_index]
test_data = data.iloc[test_index]



In [ ]:
# transform = transforms.Compose([transforms.Resize((128,128))])

In [ ]:
class CustomDataset(Dataset):
  def __init__(self,data_frame,transform = None):
    self.data_frame = data_frame
    self.transform = transform

  def __getitem__(self,idx):
    img_path=os.path.join('/content',str(self.data_frame.iloc[idx]['image_path']))
    mask_path=os.path.join('/content',str(self.data_frame.iloc[idx]['mask_path']))
    img = cv2.imread(img_path)
    img = cv2.resize(img, (128, 128))
    mask = cv2.imread(mask_path)
    mask = cv2.resize(mask, (128, 128))

    if self.transform:
      img=self.transform(img) #transform only to resize the image don't convert into tensor
      mask=self.transform(mask)
    img = torch.from_numpy(img).float()
    mask = torch.from_numpy(mask).float()
    img = img.permute(2, 0, 1)  # transpose to (batch, channels, height, width)
    mask = mask.permute(2, 0, 1)
    return img,mask

  def __len__(self):
    return len(self.data_frame)


In [ ]:
train_dataset = CustomDataset(train_data)
test_dataset = CustomDataset(test_data)



In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [ ]:
train_dataloader = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_dataloader = DataLoader(test_dataset,batch_size=32,shuffle=True)

In [ ]:
for data in train_dataloader:
    print(data[0].shape)
    break

torch.Size([32, 3, 128, 128])


In [ ]:
print(type(train_dataloader))

<class 'torch.utils.data.dataloader.DataLoader'>


In [ ]:
for i, (real_images, _) in enumerate(train_dataloader):
    bs = real_images.shape[0]
    print(bs)
    break

32


In [ ]:
class Conv_block(nn.Module):
  def __init__(self,ch_in,ch_out):
    super(Conv_block,self).__init__()

    self.conv = nn.Sequential(nn.Conv2d(in_channels=ch_in,out_channels=ch_out,kernel_size=3,padding = 1, bias = True),
                              nn.BatchNorm2d(ch_out),
                              nn.ReLU(inplace = True),
                              nn.Conv2d(in_channels=ch_out,out_channels=ch_out,kernel_size=3,padding=1,bias = True),
                              nn.BatchNorm2d(ch_out),
                              nn.ReLU(inplace = True))

  def forward(self,x):
    out = self.conv(x)
    return out

In [ ]:
class Upsample_block(nn.Module):
  def __init__(self,ch_in,ch_out):
    super(Upsample_block,self).__init__()

    self.upsample = nn.Sequential(nn.Upsample(scale_factor=2),
                                  nn.Conv2d(in_channels=ch_in,out_channels=ch_out,kernel_size=3,padding = 1, bias = True),
                                  nn.BatchNorm2d(ch_out),
                                  nn.ReLU(inplace=True))

  def forward(self,x):
    out = self.upsample(x)
    return out

In [ ]:
class Attention_block(nn.Module):
  def __init__(self,present_dim,next_dim,x):
    super(Attention_block,self).__init__()

    self.attention1 = nn.Sequential(
        nn.Conv2d(in_channels=next_dim, out_channels=x,kernel_size=1, stride=1),
        nn.BatchNorm2d(x))

    self.attention2 = nn.Sequential(
        nn.Conv2d(in_channels=present_dim, out_channels=x,kernel_size=1, stride=1),
        nn.BatchNorm2d(x))

    self.weights = nn.Sequential(nn.Conv2d(x, 1,kernel_size=1, stride=1),
                                nn.BatchNorm2d(1),
                                nn.Sigmoid())
    self.relu = nn.ReLU(inplace=True)

  def forward(self, next_l, present_l):
      next_l1 = self.attention1(next_l)
      present_l1 = self.attention2(present_l)
      psi = self.relu(next_l1+present_l1)
      weight = self.weights(psi)

      return weight*present_l

In [ ]:
class Attention_UNet(nn.Module):
  def __init__(self):
    super(Attention_UNet,self).__init__()
    self.maxpool=nn.MaxPool2d(kernel_size=2,stride=2)
    self.conv1=Conv_block(3,16)
    self.conv2=Conv_block(16,32)
    self.conv3=Conv_block(32,64)
    self.conv4=Conv_block(64,128)
    self.conv5=Conv_block(128,256)

    self.up_sample5=Upsample_block(256,128)
    self.att5=Attention_block(128,128,64)
    self.conv_upsample5=Conv_block(256,128)

    self.up_sample4=Upsample_block(128,64)
    self.att4=Attention_block(64,64,32)
    self.conv_upsample4=Conv_block(128,64)

    self.up_sample3=Upsample_block(64,32)
    self.att3=Attention_block(32,32,16)
    self.conv_upsample3=Conv_block(64,32)

    self.up_sample2=Upsample_block(32,16)
    self.att2=Attention_block(16,16,8)
    self.conv_upsample2=Conv_block(32,16)


    self.conv1x1=nn.Conv2d(in_channels=16,out_channels=3,kernel_size=1,stride=1)


  def forward(self,input):
        # encoder
        # input = input.permute(0, 3, 1, 2)
        x1 = self.conv1(input)

        x2 = self.maxpool(x1)
        x2 = self.conv2(x2)

        x3 = self.maxpool(x2)
        x3 = self.conv3(x3)

        x4 = self.maxpool(x3)
        x4 = self.conv4(x4)

        x5 = self.maxpool(x4)
        x5 = self.conv5(x5)

        # decoder + concat
        d5 = self.up_sample5(x5)
        x4 = self.att5(d5, x4)
        d5 = torch.concat((x4, d5), dim=1)
        d5 = self.conv_upsample5(d5)

        d4 = self.up_sample4(d5)
        x3 = self.att4(d4, x3)
        d4 = torch.concat((x3, d4), dim=1)
        d4 = self.conv_upsample4(d4)

        d3 = self.up_sample3(d4)
        x2 = self.att3(d3, x2)
        d3 = torch.concat((x2, d3), dim=1)
        d3 = self.conv_upsample3(d3)

        d2 = self.up_sample2(d3)
        x1 = self.att2(d2, x1)
        d2 = torch.concat((x1, d2), dim=1)
        d2 = self.conv_upsample2(d2)

        d1 = self.conv1x1(d2)

        return d1


# attention_unet = Attention_UNet().to(device)
# attention_unet

# # %%time
# num_ep = 1
# # after 30 does not improve much

# aun_lh, aun_th, aun_vh = train_model("Attention_UNet", attention_unet, train_dataloader, DiceLoss(), opt, False, num_ep)



In [ ]:
attention_unet = Attention_UNet().to(device)
attention_unet

Attention_UNet(
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv1): Conv_block(
    (conv): Sequential(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (conv2): Conv_block(
    (conv): Sequential(
      (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
 

**Sementic Matrix**

In [ ]:
def dice_coef_metric(inputs, target):
    # inputs = inputs.permute(0, 3, 1, 2)

    intersection = 2.0 * (target*inputs).sum()
    union = target.sum() + inputs.sum()
    if target.sum() == 0 and inputs.sum() == 0:
        return 1.0
    return intersection/union

In [ ]:
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=3):

        # print(inputs.shape,targets.shape)

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        # inputs = inputs.reshape(-1)
        # targets = targets.reshape(-1)

        intersection = (inputs * targets).sum()
        dice = (2.*intersection+smooth)/(inputs.sum() + targets.sum() + smooth)

        return 1 - dice

NameError: ignored

In [ ]:
# sanity check
DiceLoss()(torch.tensor([0.7, 1., 1.]),
              torch.tensor([1.,1.,1.]))

tensor(0.3050)

In [ ]:
# mask=cv2.imread("/content/DUTS-TE/DUTS-TE-Mask/ILSVRC2012_test_00000003.png")
# mask.shape

In [ ]:
import numpy as np

In [ ]:
def train_model(model_name, model, train_loader, train_loss, optimizer, lr_scheduler, num_epochs):
    print(f"[INFO] Model is initializing... {model_name}")

    loss_history = []
    train_history = []


    for epoch in range(num_epochs):
        model.train()

        losses = []
        train_iou = []

        for i_step, (data, target) in enumerate(train_loader):
            data = data.to(device)
            target = target.to(device)
            outputs = model(data)

            # print("outputs_shape",outputs.shape)



            out_cut = np.copy(outputs.data.cpu().numpy())
            out_cut[np.nonzero(out_cut < 0.5)] = 0.0
            out_cut[np.nonzero(out_cut >= 0.5)] = 1.0
            # print("out_cut_shape",out_cut.shape)

            train_dice = dice_coef_metric(out_cut, target.data.cpu().numpy())

            loss = train_loss(outputs, target)

            losses.append(loss.item())
            train_iou.append(train_dice)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


        print("Epoch [%d]" % (epoch))
        print("Mean loss on train:", np.array(losses).mean(),
      "\nMean DICE on train:", np.array(train_iou).mean())

    return loss_history, train_history






In [ ]:
# def compute_iou(model, loader, threshold=0.3):
#     valloss = 0

#     with torch.no_grad():

#         for i_step, (data, target) in enumerate(loader):

#             data = data.to(device)
#             target = target.to(device)

#             outputs = model(data)

#             out_cut = np.copy(outputs.data.cpu().numpy())
#             out_cut[np.nonzero(out_cut < threshold)] = 0.0
#             out_cut[np.nonzero(out_cut >= threshold)] = 1.0
#             picloss = dice_coef_metric(out_cut, target.data.cpu().numpy())
#             valloss += picloss

#     return valloss / i_step



In [ ]:
opt = torch.optim.Adamax(attention_unet.parameters(), lr=1e-3)

In [ ]:
%%time
num_ep = 2
# after 30 does not improve much

aun_lh, aun_th = train_model("Attention_UNet", attention_unet, train_dataloader, DiceLoss(), opt, False, num_ep)

[INFO] Model is initializing... Attention_UNet
Epoch [0]
Mean loss on train: -0.9714717261718981 
Mean DICE on train: 1.9739494362353223
Epoch [1]
Mean loss on train: -0.9751967827479044 
Mean DICE on train: 1.9770246910859974
CPU times: user 2min 8s, sys: 2.73 s, total: 2min 11s
Wall time: 2min 11s


In [ ]:
data = pd.read_csv('/content/metadata.csv')
train_index = data.loc[data['split']=='train'].index
test_index = data.loc[data['split']=='test'].index

train_data = data.iloc[train_index]
test_data = data.iloc[test_index]



transform = transforms.Compose([transforms.Resize((128,128))])


class CustomDataset(Dataset):
  def __init__(self,data_frame,transform = None):
    self.data_frame = data_frame
    self.transform = transform

  def __getitem__(self,idx):
    img_path=os.path.join('/content',str(self.data_frame.iloc[idx]['image_path']))
    mask_path=os.path.join('/content',str(self.data_frame.iloc[idx]['mask_path']))
    img = cv2.imread(img_path)
    mask = cv2.imread(mask_path)
    if self.transform:
      img=self.transform(img) #transform only to resize the image don't convert into tensor
      mask=self.transform(mask)
    img = torch.from_numpy(img).float()
    mask = torch.from_numpy(mask).float()
    return img,mask

  def __len__(self):
    return len(self.data_frame)



train_dataset = CustomDataset(train_data,transform)
test_dataset = CustomDataset(test_data,transform)



In [ ]:
import torch

# save the model to a file
torch.save(attention_unet.state_dict(), '/content/drive/MyDrive/DL_Assign/DL_attention_unet_1.pt')
